# lesson 6 configure azure sql db for scale and performance

**note:backup storage redundancy** 

geo redundant backup will replicate the storage to a paired region (e.g. US East to Us West) in case of data storage damaging

**note:billing type** 
- provisional charged based on vcore
- serverless charged based on second, shut down after 1 hr inactivity, restarted takes a little time


**note:hybrid benefit** 
- it's a Microsoft licensing program that allows customers to use their existing on-premises Windows Server and SQL Server licenses with active Software Assurance or subscription to reduce costs when running those workloads in Azure
- can only work with vcore, not with dtu
**note:elastic pool** 

- let manage multiple db resource allocation possible, so they could share resource when one not using it
- not possible for hyperscale
- can select edtu (elastic dtu) in setting
- unit price for edut compared to dtu is extra 50 %, vcore have same unti price
- if a DTU db peak at different time, elastic pool could be a good idea, otherwise it's more expensive due to unit price
- unavailable at hyperscale



### storage purchasing model
- can be changed later, except changed out of hyperscale
- change should be done at a low trafic time
- after change DMV(dynamic management view) needed to be flushed for accurate estimation
exec sq_query_store_flush
##### DTU (database transaction unit)based
- DTU can be adjusted with related S tier
- CDC (changed data captured) can't be used when having less than 100 DTB (= S3 = 1vcore)
- when using more than 300 DTU, vcore might be a cheaper option
- plan selection can based on peak usage and storage need, the latter could be estimate with average data use * db number 
1. Basic: up to 2GB
Development, testing, infrequently accessed workload
2. Standard
3. Premium
suitable when more iops (inputs output per second) is needed
##### vCore (virtual core) based
- max data storgae and core could be specified
- core decide max data storgae ceiling & tempdb size
- log space is 30% of the max data stoarge
- hardware choice affect memory, core number and storage
- iops = concurrent number of request, this can't be configured, but go up with vcore

4. general purpose: 80 vcore, 4 terabyte storage
suitable for low latency input
5. Hyperscale: 80 vcore, 100 terabyte storage
6. Business critical: 80 vcore, 4 terabyte storage, fast geo-recovery and advanced data corruption protection, free second read-only replica
Business critical has high transaction rate and high resiliency, is suitable for large number/long-running transaction

### table partitioning

- a table have 3 elements storage space, computing resource limit (exceeing might lead to timeout) and network bandwidth limit
- vertical scaling by adding disc space, processing power, memory and network connection could be a short term solution
- horizontal scaling by divifing the table up according to rule e.g. by year, it add up complexity but reduce hardware demand
- back up per partition is also faster 
- the other option is to split the table columns up

### database  sharding
- when we apply horizontal scaling on db level
- there would be a table with shard key, and where they are stored, this require more setting up 
- range strategy is diving by range e.g.years, sequential shard, data might be unblanced when there is seasonality
- hashing strategy introduce random element for distribution, it reduce hotspot bcs no cluster would be more used, but rebalancing and manging mmight be difficult, when looking up a value the results are all over places
- functional partitioning, not putting all the tables in the same db, e.g. seperate tables that needs extra security
- keeping data geographically close can reduce latency when using

**note: file group**
- can't be used in Azure SQL db (only primary file group there)
- file can be putted in file group to define partition range rules and where to store

### compression for table
- pro reduced space
- con takes extra power and time to compress and retrieve 
- can't be used on table with sparse columns
- changing compression require droping clustered index
- complete index viewed can be compressed
- system table can't be compressed
- table with different partition can be compressed differently
- *exec sp_estimate_data_compression_savings* help estimate compression effect

method:
1. no compression
2. row compression: char and nchar columns would be greatly reduced, varchar and int not so much, date types depends, tiny int not at all
3. page compression which includes (below element can't be activated seperately):
    1. row compression
    2. prefix compression: store long duplicated value in a specific column as prefix key, and generate an extra table for prefix key to long name to prevent the duplication of long char storage
    3. dictionary compression: similar to prefix compression, but instead of on a specific column, the duplication would be searched across columns and keys would be used for whole table

    - page is a set of row up to 8192 character
    - uncompressed at first, when additional rows can't be fit in, compression takes place
    - some older version SQL server on VM is this unavailable

**note heap**
haep is table without clustered index

**columnstore table**
above are row stored table. in columnstore table, columns are always compress, which could be further compressed through columnstore archival compression (but this has super high compute cost for uncompress)

### SQL data sync
- works for Azure SQL, on-prem or VM, but not with management instance
- allows sync across db
- primary key is required
- must define a hub db and multiple member dbs
- rules can be defined which direction to sync, e.g. change would be replicated from hub to member or member to hub, but not directly between members
- rule when hub and member simultaneously changed should be defined
- sync metadata db is an azure sql db whihc loacte in the same region as hub and store, should be empty at start
- for on perm or vm member db, a syn agent programm is needed 
- on azure db page syncing db could be created


# lesson 7 strategy of migration 

### migration tools

| purpose    | tool | extra function |
| -------- | ------- | ------- |
| lift & shift sql server to vm   | azure migrate    |   assess sql data estate at scale(across data center)/getting azure sql deployment recommendation, target sizing & monthly estimate  |
| non sql objects (DB2, MYSQL, Oracle, SAP, ASE) to sql/azure sql, each are independent download program | sql server migration system     ||
| sql server object to sql db/managed instance/vm/on prem     | data migration system    |upgrade SQL/assess sql data estate at scale/ performance & reliability improvement recommendation/ detect incompatibility of target/ migrate on-prime/ discover new feature on target|
| compare workloads between source and target  | database experiementation assistant    | |
| open source db (mysql, postgresql, mariadb) to azure offline/online (premium price tier)  | azure db migration service    ||

1. azure migrate  
- consists of 2 tools: discovery & assessment, server migration
**azure migrant hub** includes azure migration assistant & azure db migration service

2. sql server migration system 

3. data migration system
- for large size data azure db migration service  would be better option

4. database experiementation assistant 

5. azure db migration service 
- princing tier standard provides up to 4 vcore for free, but only support offline migration
- princing tier premium provides up to 4 vcore with 6 month free trial, support offline & online

| from    | to | online (continuous sync)    | offline (one-time) |
| -------- | ------- | -------- | ------- 
| SQL server  | azure sql db mi    | v  | v    |
| SQL server    | azure sql db    | x  | v    |
| SQL server    | azure sql vm  | x  | v    |
| mongodb    | azure cosmos db  | v | v    |
| mySQL    | azure db for mySQL  | x  | v   |
| postgreSQL    | azure db for postgresql  | v  | x  |




**reminder** if not latest SQL is needed, then VM must be used and a window version needed to be decide

**note** 
1. without downtime allowance, the only way is to do an online migration
2. if cross db dependecy exist, azure db is not a good option, as it doesn't allow cross db queries
3. for azure db migration service, allow outbond port 443 = port for https or 1434 used for UDP, then enable TCP IP protocal, next create azure SQL db instance which needs a server level firewall rule on depature db/server & contorl server permission on depature server to allow access to DMS and on target db control db permission is needed. it use exsiting full log backup instead of creating new one



